# 数据预处理
:label:`sec_pandas`

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
我们将在后面的章节中介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [13]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [14]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。


In [15]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       NaN        True      False
1       2.0       False       True
2       4.0       False       True
3       NaN       False       True


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [16]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       NaN        True      False
1       2.0       False       True
2       4.0       False       True
3       NaN       False       True


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。


In [17]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[nan, 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [nan, 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。


[Discussions](https://discuss.d2l.ai/t/1750)


# 問題1: 删除缺失值最多的列。

In [18]:
import pandas as pd
import numpy as np

# 建立一個100行10列的資料集
raw_data = pd.DataFrame(np.random.randn(10,10)) 

# 在2,4,7列中加入缺失值
raw_data.iloc[:8,2] = np.nan
raw_data.iloc[5:8,4] = np.nan 
raw_data.iloc[8:,7] = np.nan

# 統計每列的缺失值個數
missing_count = raw_data.isnull().sum()
print(raw_data)

          0         1         2         3         4         5         6  \
0  1.204215 -0.932094       NaN -0.795111  0.765896  0.401022 -0.939847   
1  0.391004  0.251712       NaN -0.796685  0.330502  0.588540  0.053198   
2  0.631579 -0.667536       NaN -0.855347  0.046683  0.289401 -1.110941   
3  0.995405  0.406753       NaN  1.786203  0.703589  0.434734  1.301045   
4 -1.927222  1.314945       NaN -1.035250 -0.666329  0.578242  0.274481   
5 -0.087796 -1.061120       NaN  0.429294       NaN  0.529372  0.792669   
6 -0.488690  1.320289       NaN -0.269533       NaN  0.517638  0.810894   
7  0.817720  0.784834       NaN  1.165735       NaN  1.319671  0.100835   
8  0.011014  2.351894  0.251870 -0.635078 -1.390251  1.376169  1.449530   
9  1.554697  2.674114  0.811193 -0.458645 -0.842525 -0.844899 -1.777039   

          7         8         9  
0  0.297002 -0.238090  2.048236  
1 -0.899903 -0.353369 -0.867384  
2 -1.706753 -0.548784  0.775180  
3  1.423195 -0.242554  0.487049  
4 -0

In [19]:
# 取得缺失值最多的那一列
most_missing_col = missing_count[missing_count==missing_count.max()].index[0] 
print(most_missing_col)

2


In [20]:
# 删除缺失值最多的那一列
raw_data.drop(most_missing_col,axis=1,inplace=True)

print(raw_data)

          0         1         3         4         5         6         7  \
0  1.204215 -0.932094 -0.795111  0.765896  0.401022 -0.939847  0.297002   
1  0.391004  0.251712 -0.796685  0.330502  0.588540  0.053198 -0.899903   
2  0.631579 -0.667536 -0.855347  0.046683  0.289401 -1.110941 -1.706753   
3  0.995405  0.406753  1.786203  0.703589  0.434734  1.301045  1.423195   
4 -1.927222  1.314945 -1.035250 -0.666329  0.578242  0.274481 -0.138544   
5 -0.087796 -1.061120  0.429294       NaN  0.529372  0.792669 -1.342143   
6 -0.488690  1.320289 -0.269533       NaN  0.517638  0.810894  1.825162   
7  0.817720  0.784834  1.165735       NaN  1.319671  0.100835  0.862077   
8  0.011014  2.351894 -0.635078 -1.390251  1.376169  1.449530       NaN   
9  1.554697  2.674114 -0.458645 -0.842525 -0.844899 -1.777039       NaN   

          8         9  
0 -0.238090  2.048236  
1 -0.353369 -0.867384  
2 -0.548784  0.775180  
3 -0.242554  0.487049  
4  0.766999 -1.104570  
5 -0.760868  0.154525  
6  0.1

# 問題2 :

In [21]:
import torch

X, y = torch.tensor(raw_data.values), torch.tensor(raw_data.values)
print(X)
print(y)

tensor([[ 1.2042, -0.9321, -0.7951,  0.7659,  0.4010, -0.9398,  0.2970, -0.2381,
          2.0482],
        [ 0.3910,  0.2517, -0.7967,  0.3305,  0.5885,  0.0532, -0.8999, -0.3534,
         -0.8674],
        [ 0.6316, -0.6675, -0.8553,  0.0467,  0.2894, -1.1109, -1.7068, -0.5488,
          0.7752],
        [ 0.9954,  0.4068,  1.7862,  0.7036,  0.4347,  1.3010,  1.4232, -0.2426,
          0.4870],
        [-1.9272,  1.3149, -1.0353, -0.6663,  0.5782,  0.2745, -0.1385,  0.7670,
         -1.1046],
        [-0.0878, -1.0611,  0.4293,     nan,  0.5294,  0.7927, -1.3421, -0.7609,
          0.1545],
        [-0.4887,  1.3203, -0.2695,     nan,  0.5176,  0.8109,  1.8252,  0.1974,
         -0.4278],
        [ 0.8177,  0.7848,  1.1657,     nan,  1.3197,  0.1008,  0.8621,  0.0638,
          0.8990],
        [ 0.0110,  2.3519, -0.6351, -1.3903,  1.3762,  1.4495,     nan,  1.3728,
         -0.2359],
        [ 1.5547,  2.6741, -0.4586, -0.8425, -0.8449, -1.7770,     nan, -1.8311,
         -0.7858]],